# Imports:

In [ ]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
import os
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from datetime import datetime

# Importing libraries fto switch to the new tab
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd
import re
import requests


# Browser preferences:

In [3]:
def ffx_preferences(dfolder, download=False):
    '''
    Sets the preferences of the firefox browser: download path.
    '''
    profile = webdriver.FirefoxProfile()
    # set download folder:
    profile.set_preference("browser.download.dir", dfolder) # you can predefine where you wanna store things in case its needed
    profile.set_preference("browser.download.folderList", 2) # 0 means to download to the desktop, 1 means to download to the default "Downloads" directory, 2 means to use the directory
    profile.set_preference("browser.download.manager.showWhenStarting", False) #I dont wanna see a pop up for each download so i swith it off
    profile.set_preference("browser.helperApps.neverAsk.saveToDisk",
                           "application/msword,application/rtf, application/csv,text/csv,image/png ,image/jpeg, application/pdf, text/html,text/plain,application/octet-stream")
    
    

    #POl:- Try the ublock_origin
    profile.add_extension('/Users/polgarcia/Documents/Master/T2/Text_Mining/ublock_origin-1.62.0.xpi') #This is using the ublock-u have to download:https://addons.mozilla.org/en-US/firefox/addon/ublock-origin/#:~:text=get%20the%20extension-,Download%20file,-Extension%20Metadata
    #profile.install_addon('/Users/polgarcia/Documents/Master/T2/Text_Mining/ublock_origin-1.62.0.xpi')


    # this allows to download pdfs automatically
    if download:
        profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf,application/x-pdf")
        profile.set_preference("pdfjs.disabled", True) #dont want the pdf viewer to open

    options = Options()
    options.add_argument('--start-maximized')#POl: i add this to start with the browser maximied.
    #options.add_extension('/Users/polgarcia/Documents/Master/T2/Text_Mining/ublock_origin-1.62.0.xpi') #to add the ublock_origin extension
    options.profile = profile
    options.set_preference('privacy.trackingprotection.enabled', True) #POL: # Enable tracking protection

    return options

'''Modify the start_up function to include a fixed window size'''
def start_up(link, dfolder, geko_path,donwload=True, window_size='1500,1080'): #POL: CHANGE THIS DEPENDING ON YOUR SCREEN:
    # geko_path='/Users/luisignaciomenendezgarcia/Dropbox/CLASSES/class_bse_text_mining/class_scraping_bse/booking/geckodriver'
    # download_path='./downloads'
    os.makedirs(dfolder, exist_ok=True)

    options = ffx_preferences(dfolder,donwload)
    service = Service(geko_path)
    browser = webdriver.Firefox(service=service, options=options)
    width, height = map(int, window_size.split(','))
    browser.set_window_size(width, height)
    # Enter the website address here
    browser.get(link)
    time.sleep(3)  # Adjust sleep time as needed
    return browser


def check_and_click(browser, xpath, type):
    '''
    Function that checks whether the object is clickable and, if so, clicks on
    it. If not, waits one second and tries again.
    '''
    ck = False
    ss = 0
    while ck == False:
        ck = check_obscures(browser, xpath, type)
        time.sleep(1)
        ss += 1
        if ss == 15:
            # warn_sound()
            # return NoSuchElementException
            ck = True
            # browser.quit()

def check_obscures(browser, xpath, type):
    '''
    Function that checks whether the object is being "obscured" by any element so
    that it is not clickable. Important: if True, the object is going to be clicked!
    '''
    try:
        if type == "xpath":
            browser.find_element('xpath',xpath).click()
        elif type == "id":
            browser.find_element('id',xpath).click()
        elif type == "css":
            browser.find_element('css selector',xpath).click()
        elif type == "class":
            browser.find_element('class name',xpath).click()
        elif type == "link":
            browser.find_element('link text',xpath).click()
    except (ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException) as e:
        print(e)
        return False
    return True

comands that i add/ were comented:
- profile.add_extension('/Users/polgarcia/Documents/Master/T2/Text_Mining/ublock_origin-1.62.0.xpi')  
    _why? i was trying to block  the sesion register pop-up with an add blocker, and the adds, 
    -this is not efective with the google pop-up (because it is some google shit) but it is efective with the adds
    -you need to download the link : https://addons.mozilla.org/en-US/firefox/addon/ublock-origin/#:~:text=get%20the%20extension-,Download%20file,-Extension%20Metadata   and add it into a folder and put the path in the comand
    - IT BLOCKS THE COOKIES 
- options.add_argument('--start-maximized')
    -usefull in two ways, 1. When we run the code is in fullscreen, that is nice becuause we will start from the same place all.
        2. it moves the pop up next to the table/list button, so we will be able to click on it and make the code more efficient (he said he would value that)
- options.set_preference('privacy.trackingprotection.enabled', True) 
    - to block adds (totaly honest dont know if works, but is nice :)

one of the addblockers blocs the cookies so it is nice.

# Walking through Booking:

In [ ]:
# lets open booking:

dfolder='./downloads'
geko_path=''
link='https://www.booking.com/index.es.html'


browser=start_up(dfolder=dfolder,link=link,geko_path=geko_path)


In [5]:
#Reject Cookies
#POL:with the new add blockers we dont need this because it blocks the cookies :) 

check_and_click(browser, 'onetrust-reject-all-handler', 'id')

In [6]:
# click on the "Where to go button"
browser.find_element(by=By.XPATH,value='//*[@id=":rh:"]').click()

## City Selection:

In [7]:
# Input Barcelona as the target city.
place = 'Barcelona'
search1 = browser.find_element(by='xpath',value='//*[@id=":rh:"]')
search1.send_keys(place)

In [8]:
# Clicking on the enter date button.
browser.find_element(by=By.XPATH,value='//*[@class="ebbedaf8ac ab26a5d2bd e33c97ff6b"]').click()

## Dates Selection:

In [9]:
# Clicking next until we find the right month.

actual_date = datetime.now()
actual_year = str(actual_date.year)

year = '2025'
month = 'marzo'
month_number = '03'
arrival_day = '01'
departure_day = '07'

# We can replace with a specific year or with the os actual year just replacing the variable year by actual_year. (Notice booking just allows you to do reservations up to one year foward.)
target_date = month + ' ' + actual_year

while browser.find_element(By.XPATH, '//h3[contains(@class, "e1eebb6a1e ee7ec6b631")]').text.strip() != target_date:
    browser.find_element(By.XPATH, '//*[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 f671049264 f4552b6561 dc72a8413c f073249358"]').click()

In [10]:
# Scrolling down 300 Pixels.
browser.execute_script("window.scrollBy(0, 300);")

In [11]:
#Clicking on the desired dates.
path='//table[@class="eb03f3f27f"]//tbody//td[@class="b80d5adb18"]//span[@class="cf06f772fa ef091eb985"]'
dates = browser.find_elements('xpath',path)
for date in dates:
    if date.get_attribute("data-date") == f"{year}-{month_number}-{arrival_day}":
        date.click()
    if date.get_attribute("data-date") == f"{year}-{month_number}-{departure_day}":
            date.click()
            break

In [12]:
# click on the "Search button"
browser.find_element(by=By.XPATH,value='//button[@class="a83ed08757 c21c56c305 a4c1805887 f671049264 a2abacf76b c082d89982 cceeb8986b b9fd3c6b3c"]').click()

In [24]:
#POl: we dont have pop ups so no need to run
# Close offer pop up if it appears.

# Wait a few seconds to see if the pop up appears.
time.sleep(1)
try:
    browser.find_element(by=By.XPATH,value='//div[@class="eb33ef7c47"]//button[@class="a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 f4552b6561"]').click()
except NoSuchElementException:
    pass
time.sleep(5)

# Loading all the hotels available:

I go for a duplicate block here because sometimes the first scroll fails.

In [25]:
#POL:we can delete this part ( is almost the same time : 20 seconds less and it clicks the same number of times ....)
#i will add the button of "Tabla"
#Put the page into TABLE format, so we have more apartments within a page
tablepath='//label[@for=":r7g:-grid" and @class="a53cbfa6de a454023a93"]'
browser.find_element(By.XPATH,tablepath).click()

In [ ]:
i=0
# Scrolling based on the height of the page.
while True:
    try:
        total_height = browser.execute_script("return document.body.scrollHeight")
        scroll_position = (total_height*10) - 200
        browser.execute_script(f"window.scrollTo(0, {scroll_position});")
        
        # Small wait to let the page load.
        time.sleep(2)
        
        # Clicking on the load more button.
        browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]').click()
        i += 1
    except NoSuchElementException:
        break

print(f'The load more button has been clicked {i} times')

In [ ]:
#POL: I think this code it takes longer to run than the one that we had, (almost 1m50s minutes)
i=0
# Scrolling based on the height of the page.
while True:
    try:
        total_height = browser.execute_script("return document.body.scrollHeight")
        scroll_position = (total_height*10) - 200
        browser.execute_script(f"window.scrollTo(0, {scroll_position});")
        
        # Small wait to let the page load.
        time.sleep(2)
        
        # Clicking on the load more button.
        browser.find_element(By.XPATH, '//button[@class="a83ed08757 c21c56c305 bf0537ecb5 f671049264 af7297d90d c0e0affd09"]').click()
        i += 1
    except NoSuchElementException:
        break

print(f'The load more button has been clicked {i} times')

# Data Extraction with request and beautifull soup


In [ ]:
#! pip install PyPDF2
import pandas as pd
import requests
import time
import os
import re
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader

# Get the current URL from Selenium
current_url = browser.current_url
print(f"Current URL: {current_url}")

response = requests.get(current_url)
if response.status_code == 200:
    print("Successfully fetched the page with requests!")

    # Parse the page with BeautifulSoup 
    #soup = BeautifulSoup(response.text, 'html.parser')
    #print(soup.prettify())
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")




In [26]:
# def get_info(hotel_url):
#     try:
#         headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
#         response=requests.get(hotel_url,headers=headers,timeout=30)
#         if response.status_code ==200:
#             info=response.content
#             return(info)
#     except Exception as e:
#         print(f"Error in link: {hotel_url}: {e}")

In [ ]:
# import pandas as pd
# import requests
# import time
# from bs4 import BeautifulSoup

# # Base URL of the page containing hotel listings
# current_url = browser.current_url
# print(f"Current URL: {current_url}")
# base_url = current_url  

# # Headers for HTTP requests
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# # Function to scrape individual hotel details from the listing page
# def scrape_hotel_list_page(url):
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     hotel_data = []
    
#     # Find all hotel containers (update the selector to match your HTML structure)
#     hotels = soup.find_all('div', {'data-testid': 'property-card'})
    
#     for hotel in hotels:
#         # Extract hotel name
#         name = hotel.find('div', {'data-testid': 'title'}).get_text(strip=True) if hotel.find('div', {'data-testid': 'title'}) else "NA"
        
#         # Extract price
#         price = hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}).get_text(strip=True) if hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}) else "NA"
        
#         # Extract rating
#         rating_container = hotel.find('div', {'data-testid': 'review-score'})
#         rating = rating_container.find('div').get_text(strip=True) if rating_container else "NA"
        
#         # Extract link to detailed hotel page
#         detail_link = hotel.find('a', {'data-testid': 'title-link'})['href'] if hotel.find('a', {'data-testid': 'title-link'}) else None 
        
#         if detail_link:
#             # Construct the full URL for the detailed page
#             detail_link = f"{detail_link}"  #POL: this not sure
        
#         hotel_data.append([name, price, rating, detail_link])
    
#     return hotel_data

# # Function to scrape hotel details from the individual hotel page
# def scrape_hotel_detail_page(detail_url):
#     try:
#         response = requests.get(detail_url, headers=headers)
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Extract the specific description or other required data (update selector as needed) a53cbfa6de b3efd73f69
#         description = soup.find('p', {'data-testid': 'property-description'}).get_text(strip=True) if soup.find('p', {'data-testid': 'property-description'}) else "NA"
#         #description = soup.find('div', {'data-testid': 'property-description'}).get_text(strip=True) if soup.find('div', {'data-testid': 'property-description'}) else "NA"   
        
#         return description
#     except Exception as e:
#         print(f"Error scraping details from {detail_url}: {e}")
#         return "Error"

# # Main script
# def main():
#     # Step 1: Scrape the listing page to get hotel data
#     hotel_list_data = scrape_hotel_list_page(base_url)
    
#     # Convert the hotel list data to a DataFrame
#     df = pd.DataFrame(hotel_list_data, columns=['Name', 'Price', 'Rating', 'Detail Link'])
    
#     # Step 2: Scrape each hotel's detailed page for additional information
#     descriptions = []
#     for index, row in df.iterrows():
#         detail_url = row['Detail Link']
#         if detail_url:
#             print(f"Scraping details for: {row['Name']} from {detail_url}")
#             description = scrape_hotel_detail_page(detail_url)
#             descriptions.append(description)
#             time.sleep(2)  # Add delay to avoid getting blocked
#         else:
#             descriptions.append("NA")
    
#     # Add the descriptions to the DataFrame
#     df['Description'] = descriptions
    
#     # Save the DataFrame to a CSV file
#     df.to_csv("hotel_data.csv", index=False)
#     print("Hotel data saved to hotel_data.csv")

# # Run the main script
# if __name__ == "__main__":
#     main()


In [ ]:
import pandas as pd
import time
from bs4 import BeautifulSoup

# Function to scrape individual hotel details from the listing page
def scrape_hotel_list_page_from_selenium(browser):
    # Use the current page source from the Selenium browser
    page_source = browser.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    
    hotel_data = []
    
    # Find all hotel containers
    hotels = soup.find_all('div', {'data-testid': 'property-card'})
    
    for hotel in hotels:
        # Extract hotel name
        name = hotel.find('div', {'data-testid': 'title'}).get_text(strip=True) if hotel.find('div', {'data-testid': 'title'}) else "NA"
        
        # Extract price
        price = hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}).get_text(strip=True) if hotel.find('span', {'class': 'f6431b446c fbfd7c1165 e84eb96b1f'}) else "NA"
        
        # Extract rating
        rating_container = hotel.find('div', {'data-testid': 'review-score'})
        rating = rating_container.find('div').get_text(strip=True) if rating_container else "NA"
        
        # Extract link to detailed hotel page
        detail_link = hotel.find('a', {'data-testid': 'title-link'})['href'] if hotel.find('a', {'data-testid': 'title-link'}) else None 
        if detail_link:
            # Construct the full URL for the detailed page
            detail_link = f"{detail_link}"
        
        hotel_data.append([name, price, rating, detail_link])
    
    return hotel_data

# Function to scrape hotel details from the individual hotel page
def scrape_hotel_detail_page(detail_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(detail_url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract the specific description or other required data
        description = soup.find('p', {'data-testid': 'property-description'}).get_text(strip=True) if soup.find('p', {'data-testid': 'property-description'}) else "NA"
        return description
    except Exception as e:
        print(f"Error scraping details from {detail_url}: {e}")
        return "Error"

# Main script
def main(browser):
    # Step 1: Scrape the listing page using Selenium
    hotel_list_data = scrape_hotel_list_page_from_selenium(browser)
    
    # Convert the hotel list data to a DataFrame
    df = pd.DataFrame(hotel_list_data, columns=['Name', 'Price', 'Rating', 'Detail Link'])
    
    # Step 2: Scrape each hotel's detailed page for additional information
    descriptions = []
    for index, row in df.iterrows():
        detail_url = row['Detail Link']
        if detail_url:
            print(f"Scraping details for: {row['Name']} from {detail_url}")
            description = scrape_hotel_detail_page(detail_url)
            descriptions.append(description)
            time.sleep(2)  # Add delay to avoid getting blocked
        else:
            descriptions.append("NA")
    
    # Add the descriptions to the DataFrame
    df['Description'] = descriptions
    
    # Save the DataFrame to a CSV file
    df.to_csv("hotel_data_MWC.csv", index=False)
    print("Hotel data saved to hotel_data.csv")

# Run the main script
if __name__ == "__main__":
    main(browser)


# Data extraction for each hotel:

In [ ]:
from selenium.webdriver.common.by import By
import time
import pandas as pd

# Creating empty lists to store the data just in case we need it.
full_names = []
full_prices = []
full_score = []
full_descriptions = []

# Creating the DF that is gonna store the data.
df = pd.DataFrame(columns=['Name', 'Price', 'Score', 'Description'])

# Sabing the original handler to be able to jump from one tab to another.
original_window = browser.current_window_handle

hotels_xpath = '//div[@class="f6431b446c a15b38c233"]'

# Finding the full hotel list.
hotels = browser.find_elements(By.XPATH, hotels_xpath)

for index in range(10):
#'''Use this for loop instead to do the full scrapping. It is gonna take a while......... (Around 4 Hours)'''
#for index in range(len(hotels)):
    try:
        # Finding the updated list on each itaration to avoid crashes since we are not working with id.
        hotels = browser.find_elements(By.XPATH, hotels_xpath)

        # Srcolling to the hotel position according to index.
        browser.execute_script("arguments[0].scrollIntoView();", hotels[index])
        time.sleep(2)

        # Clicking on the hotel.
        hotels[index].click()
        time.sleep(3)

        # Switching to the new tab using the handler.
        new_window = [window for window in browser.window_handles if window != original_window][0]
        browser.switch_to.window(new_window)

        # Getting the hotel name.
        try:
            names = str(browser.find_element(By.XPATH, '//div[@data-capla-component-boundary="b-property-web-property-page/PropertyHeaderName"]//h2[@class="d2fee87262 pp-header__title"]').text)
            full_names.append(names)
        except Exception as e:
            names = None
            full_names.append(names)

        # Getting the hotel score.
        try:    
            score = float(browser.find_element(By.XPATH, '//div[@id="js--hp-gallery-scorecard"]').get_attribute("data-review-score"))
            full_score.append(score)
        except Exception as e:
            score = None
            full_score.append(score)

        # Getting the hotel description.
        try:
                browser.execute_script("document.body.style.zoom='70%'")
                browser.execute_script("window.scrollBy(0, 400);")
                description = str(browser.find_element(By.XPATH, '//p[@data-testid="property-description"]').text)
                full_descriptions.append(description)
        except Exception as e:
            description = None
            full_descriptions.append(description)

        # Getting the hotel price.
        try:
            browser.execute_script("window.scrollBy(0, 400);")
            price = browser.find_element(By.XPATH, '//span[@class="prco-valign-middle-helper"]').text
            full_prices.append(price)
        except Exception as e:
            price = None
            full_prices.append(price)

        # Appending the data to the DataFrame.
        df.loc[len(df)] = {'Name': names, 'Price': price, 'Score': score, 'Description': description}
        time.sleep(1)
        
        # Closing the new tab and switching back to the original one.
        browser.close()
        browser.switch_to.window(original_window)

    except Exception as e:
        print(f"Error al procesar el hotel {index}: {e}")
        continue

# Printing the Final DataFrame.
df